<img src="../images/ilmudatapy-logo.png" width="350" align="center">
<br>

<center><h1>Prediksi Jenis Obat dengan Random Forest</h1></center>
<hr>

__Halo, Learners!__ Di notebook ini, kita akan mempraktekkan pemodelan <i>machine learning</i> dengan algoritma __Random Forest__ untuk kasus klasifikasi. Kita juga akan mencoba membuat visualisasinya dengan menggunakan <i>package</i> <code>pydotplus</code>.

<h2>Table of Contents</h2>
<div class="alert alert-block alert-info" style="margin-top: 25px">
    <ul>
        <li>
            Random Forest
        </li>
        <li>
            Dataset
        </li>
        <li>
            Analisis dan visualisasi data
        </li>
        <li>
            Preprocessing
            <ul>
                <li>Encoding</li>
                <li>Train test split</li>
            </ul>
        </li>
        <li>
            Modeling
            <ul>
                <li>Klasifikasi dengan Random Forest</li>
                <li>Prediksi</li>
                <li>Evaluasi</li>
                <li>Tree Visualization</li>
            </ul>
        </li>
    </ul>
</div>

<hr>
<div class="alert alert-success" style="margin-top: 20px">
    <strong>Catatan:</strong> Untuk menjalankan kode program Python di Jupyter Notebook, klik pada <i>cell</i> yang ingin di-<i>run</i> lalu tekan <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>

<div class="alert alert-danger" style="margin-top: 20px">
    <strong>Warning!:</strong> Jika ada kode program yang <i>error</i> atau output yang dihasilkan tidak sesuai, silahkan <b>Restart & Run All</b> kernel pada bagian menu <b>Kernel</b> di menu bar Jupyter Notebook, atau <b>Restart & Clear Output</b> kernel kemudian jalankan satu per satu <i>cell</i> secara berurutan dari atas ke bawah.
</div>
<hr>

## Random Forest

__Random Forest__ merupakan salah satu algoritma <i>supervised learning</i> yang dapat digunakan untuk masalah klasifikasi maupun regresi. Algoritma ini didasarkan pada konsep <i>ensemble learning</i> yang menggabungkan beberapa <i>classifiers</i> untuk memecahkan masalah yang kompleks serta meningkatkan kinerja model.

<i>Random forest</i> adalah <i>classifier</i> yang berisi sejumlah <i>decision tree</i> pada berbagai subset dari dataset yang diberikan dan mengambil rata-ratanya. Jadi, algoritma ini mengambil hasil mayoritas prediksi <i>decision tree</i> dan memprediksi hasil akhirnya. Semakin banyak jumlah pohon, akan semakin besar kemungkinan lebih tinggi nilai akurasinya serta mencegah masalah <i>overfitting</i>.

![alt text](../images/random-forest.png)

<hr>

## Dataset

Dataset yang akan digunakan adalah dataset <a href='https://www.kaggle.com/prathamtripathi/drug-classification'>Drug Classification</a> yang bertujuan untuk mengklasifikasikan tipe obat.

Kelas target:

* __Drug:__ Tipe obat

Fitur set:

* __Age:__ Usia
* __Sex:__ Jenis kelamin
* __BP:__ Level tekanan darah (Blood Pressure)
* __Cholesterol:__ Level kolesterol
* __Na_to_K:__ Rasio Natrium (Na) dengan Kalium (K)

Mari kita <i>import library</i> yang diperlukan terlebih dahulu.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Lalu <i>load</i> dataset ke dalam dataframe Pandas dengan <code>read_csv()</code>.

In [ ]:
# Load dataset

df = pd.read_csv('../datasets/drug.csv')
df.head(10)

<hr>

## Analisis dan visualisasi data

Seperti biasa, kita cek info dataframe terlebih dahulu.

In [ ]:
# Menampilkan info singkat

df.info()

Kita dapat menampilkan jumlah data pada tiap kategori di kolom <code>Drug</code> dengan visualisasi <code>countplot()</code>.

In [ ]:
# Countplot untuk kolom 'Drug'

sns.countplot(df['Drug'], palette='Set2')

Sekarang mari kita tampilkan <i>countplot</i> untuk kolom <code>Drug</code> berdasarkan <code>Sex</code>.

In [ ]:
# Countplot untuk kolom 'Drug' berdasarkan 'Sex'

sns.countplot(x='Drug', hue='Sex', data=df, palette='Set1')

Kita juga dapat menampilkan <i>countplot</i> untuk kolom lainnya.

In [ ]:
# Menampilkan countplot

fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(12, 6)) 

# Menambahkan subplot dengan indexing
ax0 = fig.add_subplot(ax[0]) 
ax1 = fig.add_subplot(ax[1])    

sns.countplot(x='BP', hue='Sex', data=df, palette='Set2', ax=ax0)
sns.countplot(x='Cholesterol', hue='Sex', data=df, palette='Set2', ax=ax1)

plt.subplots_adjust(wspace=0.3)
plt.show()

Sekarang mari kita buat <i>scatter plot</i> untuk 50 data pertama yang diberi warna berdasarkan tipe obat.

In [ ]:
# Membuat scatter plot untuk 50 data pertama

x = df[df['Drug'] == 'DrugY'][0:50].plot(kind='scatter', x='Age', y='Na_to_K', color='SteelBlue', label='DrugY');

df[df['Drug'] == 'drugX'][0:50].plot(kind='scatter', x='Age', y='Na_to_K', color='Gold', label='drugX', ax=x);
df[df['Drug'] == 'drugA'][0:50].plot(kind='scatter', x='Age', y='Na_to_K', color='Green', label='drugA', ax=x);
df[df['Drug'] == 'drugB'][0:50].plot(kind='scatter', x='Age', y='Na_to_K', color='Red', label='drugB', ax=x);
df[df['Drug'] == 'drugC'][0:50].plot(kind='scatter', x='Age', y='Na_to_K', color='Purple', label='drugC', ax=x);
plt.show()

<hr>

## Data Preparation / Preprocessing

Sekarang kita definisikan data fitur.

In [ ]:
# Mendefinisikan data fitur

X = df[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K']].values
X[0:5]

### Encoding

Selanjutnya kita lakukan proses <i>encoding</i> dengan <code>LabelEncoder()</code> untuk data bertipe kategori, seperti <code>Sex</code>, <code>BP</code>, dan <code>Cholesterol</code>.

In [ ]:
# Encoding data

from sklearn import preprocessing

enc_sex = preprocessing.LabelEncoder()
enc_sex.fit(['F','M'])
X[:,1] = enc_sex.transform(X[:,1]) 

enc_BP = preprocessing.LabelEncoder()
enc_BP.fit([ 'LOW', 'NORMAL', 'HIGH'])
X[:,2] = enc_BP.transform(X[:,2])

enc_Chol = preprocessing.LabelEncoder()
enc_Chol.fit([ 'NORMAL', 'HIGH'])
X[:,3] = enc_Chol.transform(X[:,3]) 

X[0:5]

Definisikan juga kelas targetnya.

In [ ]:
# Mendefiniskan kelas target

y = df['Drug']
y[0:5]

### Train test split

Setelah itu, kita pisahkan data latih dan data uji dengan <code>train_test_split</code>.

In [ ]:
from sklearn.model_selection import train_test_split

# Memisahkan data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

Mari kita tampilkan 10 data pertama <code>X_train</code> dan <code>y_train</code>.

In [ ]:
# Print X_train dan y_train

print('X_train')
print(X_train[0:10])
print('\n')
print('y_train')
print(y_train[0:10])

Sekarang kita tampilkan <code>X_test</code> dan <code>y_test</code>.

In [ ]:
# Print X_test dan y_test

print('X_test')
print(X_test[0:10])
print('\n')
print('y_test')
print(y_test[0:10])

<hr>

## Modeling

### Klasifikasi dengan Random Forest

Untuk membuat model <i>machine learning</i> dengan <i>Random forest</i>, kita dapat menggunakan <code>RandomForestClassifier</code>.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Membuat model dengan random forest
model_rf = RandomForestClassifier(n_estimators=80, criterion='entropy')
model_rf.fit(X_train, y_train)

### Prediksi

Mari kita menguji model tersebut dengan <code>.predict()</code> menggunakan data <code>X_test</code>.

In [ ]:
# Menguji model

y_pred = model_rf.predict(X_test)

Kemudian kita tampilkan hasil prediksi dan data sebenarnya.

In [ ]:
# Menampilkan hasil prediksi dan data sebenarnya

print('prediksi:')
print(y_pred[0:10])
print('\nsebenarnya:')
print(y_test[0:10])

### Evaluasi

Selanjutnya, kita dapat mengevaluasi kinerja model tersebut dengan <code>accuracy_score</code>.

In [ ]:
from sklearn import metrics

# Menampilkan akurasi
print('Akurasi Random Forest :', metrics.accuracy_score(y_test, y_pred))

Kita juga dapat menampilkan <code>classification_report</code>-nya.

In [ ]:
from sklearn.metrics import classification_report

# Menampilkan classification report
print(classification_report(y_test, y_pred))

### Tree visualization

Kita dapat membuat visualisasi pohonnya dengan kode di bawah ini.

In [ ]:
from six import StringIO
import pydotplus
from sklearn.tree import plot_tree

featureNames = df.columns[0:5]
targetNames = df['Drug'].unique().tolist()
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (4,4), dpi=800)
plot_tree(model_rf.estimators_[0],
          feature_names = featureNames, 
          class_names=targetNames,
          filled = True);

fig.savefig('rf_individualtree.png')

Bagan pohon yang terbentuk akan tersimpan dengan nama __'rf_individualtree.png'__ di folder yang sama dengan folder notebook ini berada.

<hr>

Copyright @ <a href="https://ilmudatapy.com/">ilmudatapy.com</a>